https://www.codeit.kr/community/questions/UXVlc3Rpb246NjA1MTZkM2MzMjkzY2E1MDBlMDZiNmI0

In [1]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import time

import pandas as pd

In [7]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
driver.get('https://www.koreabaseball.com/TeamRank/TeamRank.aspx')

driver.implicitly_wait(2)

In [41]:
def get_table(html, columns, year):

    data = []
    for row in html.find_all('tr'):
        row_data = [cell.get_text(strip = True) for cell in row.find_all('td')]
        if row_data:
            data.append(row_data)
    df = pd.DataFrame(data, columns=columns)
    df['연도'] = int(year)
    
    return df

In [43]:
# 수정
len_option = len(driver.find_elements(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlYear"]/option'))
years = list(map(str, range(2008, 2019)))

df = pd.DataFrame()

for idx in range(len_option):
    el = driver.find_element(By.ID, 'cphContents_cphContents_cphContents_ddlYear')
    option = el.find_elements(By.TAG_NAME, 'option')[idx]
    
    if option.text in years:
        year = option.text
        option.click()

        time.sleep(5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    
        # columns 생성
        header_html = soup.select_one("thead")
    
        if not 'columns' in locals():
            lst = []
            for cols in header_html.find_all('tr'):
                columns = [col.get_text(strip = True) for col in cols.find_all('th')]

        table_html = soup.select_one("tbody")

        temp_df = get_table(table_html, columns, year)
        df = pd.concat([df, temp_df], ignore_index = True)

    else:
        continue

df.head()

,순위,팀명,경기,승,패,무,승률,게임차,최근10경기,연속,홈,방문,연도
0,1,SK,144,78,65,1,0.545,14.5,3승0무7패,2패,42-0-30,36-1-35,2018
1,2,두산,144,93,51,0,0.646,0,6승0무4패,1승,51-0-21,42-0-30,2018
2,3,한화,144,77,67,0,0.535,16,5승0무5패,1승,43-0-29,34-0-38,2018
3,4,넥센,144,75,69,0,0.521,18,5승0무5패,1패,37-0-35,38-0-34,2018
4,5,KIA,144,70,74,0,0.486,23,4승0무6패,1패,43-0-29,27-0-45,2018


In [44]:
df.tail()

,순위,팀명,경기,승,패,무,승률,게임차,최근10경기,연속,홈,방문,연도
93,4,삼성,126,65,61,0,0.516,18,5승0무5패,1승,33-0-30,32-0-31,2008
94,5,한화,126,64,62,0,0.508,19,6승0무4패,2승,35-0-28,29-0-34,2008
95,6,KIA,126,57,69,0,0.452,26,5승0무5패,1승,30-0-33,27-0-36,2008
96,7,우리,126,50,76,0,0.397,33,6승0무4패,1승,26-0-37,24-0-39,2008
97,8,LG,126,46,80,0,0.365,37,4승0무6패,1승,24-0-39,22-0-41,2008
